In [153]:
import pandas as pd
import numpy as np
import gc
import torch
import os
import sys
import math

In [154]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [155]:
device

device(type='cuda')

In [156]:
!nvidia-smi

Sun Sep 29 02:10:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM3-32GB           Off |   00000000:05:00.0 Off |                    0 |
| N/A   29C    P0             45W /  350W |       3MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [157]:
!ls data

logs_df_2024-07-08.parquet  logs_df_2024-07-24.parquet
logs_df_2024-07-09.parquet  logs_df_2024-07-25.parquet
logs_df_2024-07-10.parquet  logs_df_2024-07-26.parquet
logs_df_2024-07-11.parquet  logs_df_2024-07-27.parquet
logs_df_2024-07-12.parquet  logs_df_2024-07-28.parquet
logs_df_2024-07-13.parquet  logs_df_2024-07-29.parquet
logs_df_2024-07-14.parquet  logs_df_2024-07-30.parquet
logs_df_2024-07-15.parquet  logs_df_2024-07-31.parquet
logs_df_2024-07-16.parquet  logs_df_2024-08-01.parquet
logs_df_2024-07-17.parquet  logs_df_2024-08-02.parquet
logs_df_2024-07-18.parquet  logs_df_2024-08-03.parquet
logs_df_2024-07-19.parquet  logs_df_2024-08-04.parquet
logs_df_2024-07-20.parquet  logs_df_2024-08-05.parquet
logs_df_2024-07-21.parquet  logs_df_2024-08-06.parquet
logs_df_2024-07-22.parquet  video_stat.parquet
logs_df_2024-07-23.parquet


In [158]:
logs_df_2024_08_06: pd.DataFrame = pd.read_parquet('./data/logs_df_2024-08-06.parquet')[:10000]

In [159]:
video_stats: pd.DataFrame = pd.read_parquet('./data/video_stat.parquet')

In [160]:
logs_df_2024_07_08: pd.DataFrame = pd.read_parquet('./data/logs_df_2024-07-08.parquet')

In [161]:
del logs_df_2024_07_08
gc.collect()

262785

In [162]:
logs_df_2024_08_06_with_vstats: pd.DataFrame = pd.merge(
    logs_df_2024_08_06,
    video_stats[['video_id', 'v_duration']],
    on='video_id',
    how='left'
)

In [163]:
logs_df_2024_08_06_with_vstats

,event_timestamp,user_id,region,city,video_id,watchtime,v_duration
0,2024-08-06 21:14:39+03:00,38aed18b-57e3-4e6b-a20f-9345f158196e,e8a62ae8-802d-417c-92e2-32f2cfdda958,4faaf176-823e-43d6-b492-c5e1fb6ec73d,4be0ce24-d33f-466b-a896-3e464911bdaf,0,9038.998
1,2024-08-06 12:12:40+03:00,1fc19927-d43a-43ae-9044-dc096595d822,20bba10e-ac0a-4578-a520-04fa8a8334ef,eebf23d2-2358-4af1-a88e-819fb96ea227,6c960e61-dd0f-4cb8-ba5c-49029fa07301,2222,2501.040
2,2024-08-06 18:38:28+03:00,712fa048-e6d7-47df-b8c9-26fe232e0547,c431eec5-716c-4e79-a9d0-96034b75a83f,385196d1-ced6-4e87-8137-733fe92a3d9e,fd216d57-6fc4-4eb1-9279-44eb7fd68712,1051,1339.102
3,2024-08-06 21:54:57+03:00,b7e2814e-87a6-4645-b58f-3ba74b5cfa23,22f32a19-5dfd-4522-b819-5e4449cb1322,5f11ec8c-2e13-4e1e-8c7f-7ffd93edeb98,33fff03e-1025-4541-8339-e40e87738c73,9,3541.505
4,2024-08-06 22:20:50+03:00,fe3a357d-9d38-4cb0-8dee-4eac0c51ea23,ed19c87e-af48-433e-970f-25a7440935f0,224ec1f1-b705-4d72-8aa9-970e1fb20168,b7f9debc-7770-4524-a4ab-f6fa7c3c830f,1333,1626.410
...,...,...,...,...,...,...,...
9995,2024-08-06 20:41:56+03:00,aa45bf8d-f5b0-4505-adc5-851aedd2e6bc,ba6f0260-b2a7-4867-a757-f0a6f3cb8d24,ac8c21f1-cf6a-4aca-b100-02199e470421,9e00dd05-4afc-4d39-a744-dcc5687fa4ab,2,1844.200
9996,2024-08-06 11:57:12+03:00,bf327a57-623b-44c4-95ed-c3ced683e548,b899f98b-1c84-43b8-be8a-255620e7b284,759895d7-b09d-4058-bc51-397ffa7ed949,15b34f0c-88c6-4977-a51a-0a921a1fa482,0,13.500
9997,2024-08-06 19:15:52+03:00,0f2084a4-7948-4885-9507-1086fc3a5a41,3d1b5b60-5801-41b1-9db7-da52641aafee,cbebad56-6612-474f-82d6-0330f7ec94cc,22abae7d-0305-44b7-87b0-e827e48c47d4,0,442.248
9998,2024-08-06 09:17:20+03:00,1f8c7eb3-41b9-46c2-b065-f694d15bd4cf,c431eec5-716c-4e79-a9d0-96034b75a83f,385196d1-ced6-4e87-8137-733fe92a3d9e,bd5c63e5-4361-408e-8553-51b15ef29eab,481,5900.096


In [164]:
logs_df_2024_08_06_with_vstats['watchtime_is_long'] = logs_df_2024_08_06_with_vstats.apply(
    lambda row: 1 if (
        (row['v_duration'] > 300 and row['watchtime'] / row['v_duration'] > 0.25) or
        (row['v_duration'] <= 300 and row['watchtime'] > 30) or
        (row['v_duration'] <= 30 and (row['watchtime'] != 0 and row['v_duration'] != 0 and row['watchtime'] / row['v_duration'] > 0.75))
    ) else -1,
    axis=1
)

In [165]:
from tqdm.auto import tqdm
tqdm.pandas()

In [166]:
logs_df_2024_08_06_with_vstats.head()

,event_timestamp,user_id,region,city,video_id,watchtime,v_duration,watchtime_is_long
0,2024-08-06 21:14:39+03:00,38aed18b-57e3-4e6b-a20f-9345f158196e,e8a62ae8-802d-417c-92e2-32f2cfdda958,4faaf176-823e-43d6-b492-c5e1fb6ec73d,4be0ce24-d33f-466b-a896-3e464911bdaf,0,9038.998,-1
1,2024-08-06 12:12:40+03:00,1fc19927-d43a-43ae-9044-dc096595d822,20bba10e-ac0a-4578-a520-04fa8a8334ef,eebf23d2-2358-4af1-a88e-819fb96ea227,6c960e61-dd0f-4cb8-ba5c-49029fa07301,2222,2501.040,1
2,2024-08-06 18:38:28+03:00,712fa048-e6d7-47df-b8c9-26fe232e0547,c431eec5-716c-4e79-a9d0-96034b75a83f,385196d1-ced6-4e87-8137-733fe92a3d9e,fd216d57-6fc4-4eb1-9279-44eb7fd68712,1051,1339.102,1
3,2024-08-06 21:54:57+03:00,b7e2814e-87a6-4645-b58f-3ba74b5cfa23,22f32a19-5dfd-4522-b819-5e4449cb1322,5f11ec8c-2e13-4e1e-8c7f-7ffd93edeb98,33fff03e-1025-4541-8339-e40e87738c73,9,3541.505,-1
4,2024-08-06 22:20:50+03:00,fe3a357d-9d38-4cb0-8dee-4eac0c51ea23,ed19c87e-af48-433e-970f-25a7440935f0,224ec1f1-b705-4d72-8aa9-970e1fb20168,b7f9debc-7770-4524-a4ab-f6fa7c3c830f,1333,1626.410,1


In [167]:
logs_df_2024_08_06_with_vstats = logs_df_2024_08_06_with_vstats\
    .iloc[
        logs_df_2024_08_06_with_vstats.groupby(by=['user_id', 'video_id'])['watchtime'].idxmax()
    ]

In [168]:
logs_df_2024_08_06_with_vstats = logs_df_2024_08_06_with_vstats.drop(
    columns=[i for i in logs_df_2024_08_06_with_vstats.columns if i not in ['user_id', 'video_id', 'watchtime_is_long']]
)

In [169]:
logs_df_2024_08_06_with_vstats

,user_id,video_id,watchtime_is_long
9991,0007638d-c723-42af-bafb-71672ad3f02b,a1f0defc-fe5a-4f31-a99e-f2d9ef936e5b,1
5090,000af647-20d8-4d93-b27a-d73b1753d2de,85372f69-97fc-485d-ae4a-91e36aeb4db3,-1
135,00114cf9-7156-45b6-9f2a-2808e4e9b934,c8eb8489-cba8-4e60-b376-412413f40304,-1
3011,00268e85-beb7-464b-886e-a6b214856e13,fdabd948-f7c2-4d29-b6a4-d6d3fdb47feb,-1
3490,00351035-a914-4d86-9d5e-bdb5e9b0b4e0,2a27a0e7-a03e-4e6c-803f-04ca0b80d8e4,1
...,...,...,...
6919,ffe21d6f-ce09-48d8-a118-da320e6ff02b,00a05c00-03c8-400b-aacb-81760b50ea43,1
9437,ffe950c0-0732-4e05-a434-68cfa23a9a4b,e0eedb0a-2468-4193-8b76-2c41fd9af529,1
6074,fff19441-08b3-478a-a578-300b115f18db,d3220e81-b748-432b-84a6-9933d6bf89d3,-1
9405,fff45ceb-55dd-4c18-9bf8-c0f4c654fa24,594bf8fa-19d9-4d6e-80e9-eeafd7acbe1e,1


In [170]:
logs_df_2024_08_06_with_vstats.to_parquet('./tmp.parquet')

In [171]:
logs_df_2024_08_06_with_vstats['item_id'] = logs_df_2024_08_06_with_vstats['video_id']
logs_df_2024_08_06_with_vstats['response'] = logs_df_2024_08_06_with_vstats['watchtime_is_long']
logs_df_2024_08_06_with_vstats = logs_df_2024_08_06_with_vstats.drop(columns=['video_id', 'watchtime_is_long'])

In [172]:
logs_df_2024_08_06_with_vstats['response'] = logs_df_2024_08_06_with_vstats['response'].replace(-1, 0)

In [173]:
from sklearn.model_selection import train_test_split
train_logs_df_2024_08_06_with_vstats, test_logs_df_2024_08_06_with_vstats = train_test_split(logs_df_2024_08_06_with_vstats)

In [174]:
train_logs_df_2024_08_06_with_vstats

,user_id,item_id,response
9829,be09c270-d107-4e65-bdc1-e18346d054f6,7d480a3e-7b10-4fcd-9eab-91a8bbcd2ef3,0
138,33b37565-f9d2-4a01-89f0-efcee3655c54,a7bf6838-a812-4eb6-aa34-b37ac06184a6,1
7623,ad696bf4-70c9-47ea-b7a7-9b85daade668,70c1cf85-d20b-412a-b47d-5eeaec7c5a59,0
8654,bea130b2-e33b-4a47-8e8d-83a418608f07,681f5fa7-e40e-41aa-8506-787421ed6761,0
9299,074f4c86-143f-4ee2-a126-c99264e32326,c59e9c63-b2c2-4589-95ba-9f655623cdf9,0
...,...,...,...
3596,7ff44127-a83e-484e-ad96-e0421a0a9883,95f9c899-4aea-4459-bdb1-870d0f639906,0
9038,5a29f479-f442-4f80-a6c8-6099442ef862,107d5d77-138b-49c8-b54a-bfea9e9b0ab1,1
514,1beec856-c4f3-44d6-8601-a37366a5c8a4,2e65c575-0f15-46a1-80f4-bc04aade8e98,0
9153,5192774e-5fb1-4d71-9b67-f8af0c06dae2,a973318c-a98a-475e-b516-c0aa5e2dc991,1


In [175]:
video_ids = logs_df_2024_08_06_with_vstats['item_id'].unique()
video_info: pd.DataFrame = video_stats[video_stats['video_id'].isin(video_ids)]

In [176]:
video_info = video_info.drop(columns=['description'])

In [177]:
from sklearn.model_selection import train_test_split
train_logs_df_2024_08_06_with_vstats, test_logs_df_2024_08_06_with_vstats = train_test_split(logs_df_2024_08_06_with_vstats)

In [181]:
from mab2rec import BanditRecommender, LearningPolicy, NeighborhoodPolicy

In [189]:
recommenders = {
    'Random': BanditRecommender(learning_policy=LearningPolicy.Random()),
    'Popularity': BanditRecommender(learning_policy=LearningPolicy.Popularity()),
}

In [190]:
recommenders

{'Random': <mab2rec.rec.BanditRecommender at 0x7f0046010c70>,
 'Popularity': <mab2rec.rec.BanditRecommender at 0x7f00460132b0>}

In [191]:
from jurity.recommenders import BinaryRecoMetrics, RankingRecoMetrics

metric_params = {'click_column': 'score', 'user_id_column': 'user_id', 'item_id_column': 'item_id'}

In [197]:
top_k_list = [3, 5, 10, 15, 25, 30]

metrics = []
for k in top_k_list:
    metrics.append(BinaryRecoMetrics.AUC(**metric_params, k=k))
    metrics.append(BinaryRecoMetrics.CTR(**metric_params, k=k))
    metrics.append(RankingRecoMetrics.Precision(**metric_params,  k=k))
    metrics.append(RankingRecoMetrics.Recall(**metric_params, k=k))
    metrics.append(RankingRecoMetrics.NDCG(**metric_params, k=k))
    metrics.append(RankingRecoMetrics.MAP(**metric_params, k=k))

In [198]:
video_info.rename(columns={'video_id': 'item_id'}, inplace=True)

In [199]:
from mab2rec.pipeline import benchmark

reco_to_results, reco_to_metrics = benchmark(recommenders,
                                             metrics=metrics,
                                             train_data=train_logs_df_2024_08_06_with_vstats,
                                             test_data=test_logs_df_2024_08_06_with_vstats,
                                             item_features=video_info)

/usr/local/lib/python3.10/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/user1/.local/lib/python3.10/site-packages/jurity/recommenders/precision.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged = merged.fillna(False)
/home/user1/.local/lib/python3.10/site-packages/jurity/recommenders/map.py:131: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged = merged.fillna(False)
/home/use

In [200]:
train_logs_df_2024_08_06_with_vstats

,user_id,item_id,response
5042,b76eaa96-cb15-4317-9be3-49d6e24c3338,aafc5f4f-e284-4577-8a03-d2ac2e58cfe3,1
108,20e5483b-77db-4887-bc94-740716ec3b53,6861d908-fb14-4c05-b20e-485652af05c9,0
103,cdbceb5e-4ddd-4df2-994a-13b8617fba12,689db4cc-0868-4770-ae1c-c4688927ed7d,0
3350,b0cc5138-82ed-4be9-a0f8-c4197c63543a,d2e62e32-8056-4c7f-a77e-bb2c97e25338,0
1013,fc8fc30d-347c-41f6-970b-30be915e102c,3ed989b7-3e74-4aae-9afd-17eaea8a3e89,1
...,...,...,...
4277,3cb2f7d2-2597-4e69-93d9-7abc55ab9f63,83eaff50-947d-45d9-84fb-4eb44da8c06f,0
9824,089f1406-ae99-4d6d-a762-609679923e17,ddface47-5724-4e38-a4f0-dc4f2adc3031,1
9307,c46234f6-f69f-42be-96a4-3e32a8bacbb4,45657741-bc2c-4749-acf2-a68b0972fea6,0
8103,967513ac-10e6-4156-853e-02c349af091c,fba64c7c-ee47-49a7-b897-ae16dfadf0e4,1


In [ ]:
reco_to_results

In [ ]:
os.listdir()